# Application Setup/Initialization

In [1]:
# Import Dash components
import dash
import dash_html_components as dhtml
import dash_core_components as dcc
from dash.dependencies import Input, Output

# Jupyter dash to view/deploy dashboard in notebook
#from jupyter_dash import JupyterDash

# Import various libraries
import pandas as pd
import mysql.connector as sql
import time
from time import localtime
from pytz import timezone
import datetime
import plotly
from env_canada import ECData

In [2]:
# Create dash application
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)

# Dataframe Updating Functions

In [3]:
# Forecasting Dataframe Updating
def build_forecasting_df():
    mydb = sql.connect(host="localhost",
                       user="data_loader",
                       passwd="data_loader_password",
                       database="dashboard")
    
    sql_query_for_line = "SELECT * FROM forecasting"
    
    forecasting_df = pd.read_sql_query(sql_query_for_line, mydb)
    
    return {"forecasting_df": forecasting_df}

In [4]:
forecasting_df_dict = build_forecasting_df()
forecasting_df = forecasting_df_dict["forecasting_df"]

# Application Layout

In [5]:
# Global Layout Settings
layout = dict(
    autosize=True,
    automargin=True,
    margin=dict(l=30, r=30, b=20, t=40),
    hovermode="closest",
    plot_bgcolor="#F9F9F9",
    paper_bgcolor="#F9F9F9",
    legend=dict(font=dict(size=10), orientation="h"),
),

pretty_container = {
    "borderRadius":"5px",
    "backgroundColor":"#f9f9f9",
    "margin":"10px",
    "padding":"15px",
    "position": "relative",
    "boxShadow": "2px 2px 2px lightgrey"
}

mini_container = {
    "borderRadius":"5px",
    "backgroundColor":"#f9f9f9",
    "margin":"10px",
    "padding":"15px",
    "position":"relative",
    "boxShadow":"2px 2px 2px lightgrey"
}

container_display = dict(
    display="flex"
)

In [6]:
# Create app layout
app.layout = dhtml.Div(
    [
        dhtml.Div(
            [
                dhtml.Div(
                    [
                        dhtml.Img(
                            src="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIQEhUSExMVFhIVGBgYGBgVEhIXGBcXFxcYFhkVFxgYHSghGB4lHRcaITEhJSkrLi8uFx8zODMtNygtLisBCgoKDg0OGhAQGy4lICUtLyw3LTcwKzEtKy01LTUuLi03MjU2KzIrLzU1Ly0tLTU1Ky4tLSstLS0vLS0tLTUtLf/AABEIAPoAygMBIgACEQEDEQH/xAAcAAEAAgIDAQAAAAAAAAAAAAAABgcFCAEDBAL/xABJEAABAwIDBAYGBQgIBwEAAAABAAIDBBEFBiEHEjFBEyJRcYGRIzJhobHBCBRCUrIXM1NygpLT8BVUYoOTosLRNEOjs8PS8ST/xAAaAQEAAwEBAQAAAAAAAAAAAAAAAwQFBgIB/8QAMhEBAAIBAgQEAwYHAQAAAAAAAAECAwQRBRIxQRMhUWEykbEicYGh0fAzQlKSweHxI//aAAwDAQACEQMRAD8AvFERAREQEREBERAREQEREBERARdc8Qe0tPAjkqxfjNRQzGIPJY03LdNb66359yjvk5Oq5pNHbU80Unzjt6rSRRnCM5QTAB/o3nSx1+CkrHAi4NwvVbRbogzYMmGdrxs5REXpEIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICrjaLRbkzZBf0nH9lob8grHUU2h05dBv8AJl/M2UWaN6S0OF5OTU19/JWqymE49PTEBjzuX1B1J81iwioRMx0dffHW8cto3hadHmdoa3p2lm/6rvWB7wBopBHIHC7TcHmFD8lCOppXQPG8GdvHrkk2PesbXxVWFv3o3F0BJNuPDlrw4q7GSYrFp84ctk0eO+W2Kk8tonpPSfu9FiIo/gOaoamzSdyT7pv8eAUgU1bRaN4ZuXDfFblvG0iIi+oxERAREQEREBERAREQEREBERAREQFhs4R71HKP1PxtWZWPx7/h5O4fiC82+GU2nnbLSfePqptcLl3E95+K4WY7xLtnNRuyvb98/AX+SsKqpmStLHtDmnkf50VX5HkAqowTqSbfuOVqq9p/OjlOMRy6nmjvESqXMuCPo5Lj1HHqubcefYsvlnOJYRHObtPB2mnsU0xrDW1MRjdz5qn62ldE90bhYg+7kobxOK29ejR0mTHr8Ph5Y+1Hz+9dkUoeA5puDwK+1V+U8yupnCOQ3iOgvy7NVZ0UgcA4G4PAqzjyReGHrNHfTX2np2l9IiKRTEREBERAREQEREBERAREQEREBY/MBtTyX7B+ILILCZzm3KOU2v6v42k+4Lzedqym09ebNSPePqqZ3E95XCIsx3iQZHjBqoyeIJt+45Wqq22dwb0rnfcPyt81ZKvaePsOT4zbfUbekQKB7RMKtado7d/22AA/n2KeLHY/RdPA+Ptsf3SD8lJkrzVmFTRZ5w5627d/uU2prkPHy0inkOh9UnkB2lQs8SOwkeRXLXWII0t2dnYqFLTWd4dfqdPXPjmlv+LzRYvLeJipga/7XMeOnuWUWjE7xvDiMlJx2mtusCIi+vAiIgIiICIiAiIgIiICIiAoftGqd2JrOT7jjrpbkpgqtz1XdLUuYPVZax/ZF/fdQ57bUaXCcXiaiJ9PNHAiLkDUDtIHnoqDsFh7N6XdjfJb193Xu3v91MVi8tURgp2RnlfyJ09yyi0scbViHD63L4ue9vcXBC5Re1VTOOwdHO9nYvAsznEAVsoHKw82h3zWGWZbytLvNPbmxVme8R9Ew2dVu7K6M/b0HsDQXKxVTuWZujqo3eHnoriVzTzvXZzfGsfLni0d4ERFOxxERAREQEREBERAREQEREHgxzEBTwvkPIcuOuipyWQucXEkkk6njbkpPnrGumk6JhO4zj2OuBfTv08FFVRz35rbR2dbwnS+Di5rdbfTsLMZUoOnqGtI6g4+wjVYdWZkLCuhh6Rws+TUg8raLzipzWT8Q1HgYJnvPlCUoiLQcWIiIKizcb1kp7bHxtb4ALDr2YtP0kzn9q8azLTvMu9wV5cVa+kQ9FAbSxkffb+IK7VSND+cj/XZ+IK7lZ03SWDx34qfj/gREVpgCIiAiIgIiICIiAiIgKL5yzEIGGKM+lcOPJvf7V6c0ZiZSssDeQ6AKrqqodK8vebuKr5su3lHVtcM4dOSYy5I+z29/wDTrJvqeJ1PeuEWVy/gb6t9hcRj1nfIdqpxEzO0OlyZK46za07RD1ZRwM1Moc4eiYdfaRqCO3VWq1oAsOAXmw6hZBGI2CzQvUtDFj5Icdr9ZOpyb9o6CIikURY/HazoYHvPKw8zb5rIKGbRcR3Y2wj7d973W+a8ZLctZlZ0eHxc1ae/5K+cdSe0k+ZXCIs13L04YzemjH9pp8nBXYqgyrCX1cTe2/uBd8AVb6uaaPKXNcdt/wClK+37+giIrLCEREBERAREQERdFZWMhaXvcABr/wDAj7ETM7Q71FM0ZsZBeOI70nMj7PtWDzFnJ0oMcPVZzdpr7FEtTrYknnYlVcmftVv6HhH8+f5fq+55nSOLnm7iuv8An2rKYdgFROeozq8zwt5qcYFkyKGzpOu8cOVvLioaYrWamo1+DTxtM7z6R+/JFsuZVkqSHvBZF7RxHZZWTQUMcDAyNoDQvQ1oGg0C8VdNMDaOPeHbdvzKt0xxSHNanWZdXbaZ2j07PciwE9dWt9WnLv2oh/qXimxyuYLupbftNPwJC9TkiEVdHe3Sa/3QliKBPz5K02NOSRxGo94Bsu2n2ggnrwhv96T/AKV58anqnnhWq235fzhNJ5gxpc42AVPY7iJqJnScjy7CNPks7mrNf1hgii0Yb7xB1PC3dz81ElXz5ObyhscK0NsMTkvHnP5QIiW9+nmq7ZS/ZzQ70rpbGzDobadZpHHxVjLAZLw/oaZpIs93HwJAWfWjiry0hxfEc3i6i0x0jy+QiIpFEREQEREBfL3gC5Nh7Vg8SzI1juiib0kt/V4eR5rGSYRWVp9NJ0TL+qBf4EKOb9o81vHpJ25ss8se/X5dXfjmc4oerEOkeNDbl4c1EZIazEHb9i4X5WFvZbgp7h2VqaGxDbu5k8+8LNAW0Xicdr/FK3TW4NN/ApvP9UoLhmQuDpX6c22HloVJ6DL9PB6jPMk+7gsoikrjrXpCpm12fN8VvwERF7VBERAREQdNRSskFnNB5e23esBX5MppAd1u67t1PxUlReZrE9YTYtRlxfBaYVdimTZ4fUvI3mdBbw4qOOBBsQQfaLK9Fgcw5ZiqmkgbsltHD/ZV76fvVtaXjU78uaPxVQsvlfCzUztaR1BqSORGoXgrKJ8UhieOve3nwKs7J+DfVoRvAdI7V1vcocWPmt5tHiGsjDh3rPnPT9WdY2wA7F9Ii0HHCIiAiIgIiIOuKBrfVaB3BdiIj7M79RERHwREQEREBERAREQEREBERBjanBopJmTEDebflx0sskiL5ERD1a9rRETPQREX15a8Zjx/NGHt36icsZe1+jpHe9sdio1+VnGf64f8Cm/hrYHag5gw2cvjD29W4Lb2u4dYdhHG/Ky1KfxNu0oLAwraLj9VIIoapznuNhaCmt4no9Fc+zNmMWldijwRdoiFoge1xtG0acOJ7fHC7BcBZFR/WbAumtrxI3Sb93LyVpoCp7arjeOUtQ99M/oqNoaGEMhdvdRpcTvtJvvFw48ANFcK+XsDhYgEdhF0GqztrGNDjWEf3FN/DUjyZmzMWISt6Go6RjXDeDoqZoIHWLXHcBAI9o48Vjs/ZXj/AKcNNBZrXujO7yaCxjiB2C5NgtisFwmKkhZDE0NY0chbUm5PiSSghm1OoxiPcfhzi2NrXGQhsLr29j2kql5NquNtNnVbgRyNPTA/9tbVELWLbfSxtxR7Im9YtY527rxHAAIJvs12tunkbT1hu5w0lIYNfaGgCyuYFaQRSFhDgbEG47wtodkudhiVOI5CPrEQAd/a42I7gAgny6axzwxxYLvsd0aceXFdyINcc0ZtzJQO9PO5jSSR6GmIAvpc9GsJDtUxt5DW1bi46ACCmJP/AE1e216BjsKqC4A26OxPEXmY02PcT5qudgOVmyvkrZGNIjO7FfU3NwdOFvagnGQZsYdRTyVjuknfY04IgbYWIPqgAa69ZV1mXH8z0DTJPOWR7xAPR0juenBmq2FAUY2mPY3DKkyNDmhg0PPrtQa9/lZxn+uH/Apv4azWA5vzJXBzqeoLw3j6GlH/AI1WL7XPeVsvsIw7o8PEtiOmtxFr7hcL+8oIDiOO5op4zJJOQwcT0VL/AA1jsF2xYlC+9RI6YfcMcLPwsBWyxYDxA8lrTtwwKOmrOkjFmy/ZFrCzReyC7si54p8VjBYd2YAF8d/VNtbHmpUtS9l2LPpcQie0m1nAgW6wItrdbZtNxdByiIgrbbxiHR4eYbkdNfhz3C028yFrVC3ec0dpA8yro+khWlz6WIG24JLi/Hf6O1xytue9VBhFG+aVrI/XOre8ahBttkKg+r4fTxWtus+Lifms+uunjDWtaOAAA7gLLsQFw42F+xcrqqnAMcTwDT8EFFUMwq80iQDeYdCTy3ISRw9jAPFX0qI2H0xnr6ioOnRuIsbX9VzNbc9Ve6AqB2f08eJYzLUSjeYG8r2LiDxt8Vd2P131enlmvbcaTfT5qpfo6Yfb65KRoTGI+OgvKHd/BvkgrbaRk5+F1O5qYn6sebdbt05LGZPzFJh9SyeNxG6dQCbOB0O8OYtyWz+0DKjMTpnRkDpQD0btLgm1wL9tgtTcQonwSOjkaWuaeBFu42Qbj5exmOtgZURG7Xi/ceYWSWtuxnPQopjBO70Eg4k6M3b2+JWyLTcXHAoK/wBuVW1mFSxn/mlgHe17Xj3geS79itJ0eEwEt3XP37j9WR7AT37t/FRL6QlaXGkpxqDJdzWtJeSbWAA491r3VrZeougpoorW3W8DyJ6xB8SgyKrLbxinRUPQfpr/AOQt/wDZWaqG+kfVb0tNGD+bDwdePSbh924PNBTUIu5o9oHvW3uQMPNNQwxHkCRrycS4W9mq1IwykfNI1kbC95Nw0c7clsJXYzmQDqUAYLAaSUj7Hts1zj7rILNrKtkLC+Rwa0C+pHuWqG0rMX1+tlka68W91B2WaGke6/ivRnrMGKzO6Otc4WNwA0Nty4tAC7NmuEYbUS2rp90k2ay1gSeZdyQZLYrlSSqq2zub6CK+9cEb1xpbtWzAC8WEYZDSxNigYGRtGgBJ07yST4le1AREQaybd8SE2JOY29o2MB0t1rG9vYvHsXw8z4pELDcaHl178A29u9YjaA9zq6UvvvaXup79HamDqmZ/No+IIQbAoiICwGfK76vQVEt7brRr+s9rfms+qp29Y8I6ZtG3V85seFmgEWvzug+fo90m7TVExGssgNyOIG8rYUS2WYM6jw2GN/5zrFx7QXuLf8u6FLUEG2y4g6HC5g0gOkAbrf1bi9rHiujYhRCPC4ni29JfetfTdc4AG44qNfSHrHAUkA9WQv3/ABdHu/ByszJ+GClpIohyF/3jvfNBmVTG3bJQez6/C3rNsJA0XJuQGm3Zf4q511zwtkaWOF2kWIQaRtcWm/MG/iFsnsbzyK2EU8rvTRAAE/b46DuAHmqj2qZNdhtSS0f/AJ5D6M9ugLr9mtx4KM5exqWhnZPEbPYfA9t0FqZ7vW5hbScoyzzMEb9PP3K+Ym2AHYAFr7stk/pHGvrzm2LS42tpcwvbc+JWwiAtXtt+I9NicrR6jAwDvDADbxWz0xs0n2H4LTTMeJ/Wqh033rcUEi2PU3SYpTgtuy7gT2WY5wPm23itrFQn0dcPEkk8xH5stsdOLg4D3XV9oMJmvLUOIwOilaLkdV3MHl4exagVUTonlpG64HxC3WqJmxtL3GzWgk+C0xx2tM8z5DxOh8EF9bCs3SVUTqWZ1zCBuE8SPu9wCtha9fR5hcaqRwGjQb+y7SAthUBERBrxt4yoYakVcYPRyjrC97FtrkeY0UQ2cZtOF1TZDcxO0eB2HS/gtrq+ijnjdFI0OY4WIKpjMuw0ukc+llaGEaRlu7Y95JB9yC3sDxqCtjEsDw9pAOnK/b/PJZFa2wbIcZheHRiLq+qTM34clkJ9nuYJGljuh3Tx9KxBbGc89UuGMdvuDpgOrHe1za4BNjbTVVNkjCJ8exF1fUg9CHBxBF26N3WtA5eqNOzXXnk8rbEHh7X1krd1vGIN3rk9jg4BvkVc2F4fHTRMhibuxsFgO8kk+JJPig9IFtBwXxUTtjaXPcGtHMmy7FW21DJFXikjOikDY2crgHhqgqHavmpuIVx3CDDGQGnXlobhbG5UzBDXQMkic31Rdo0tbTQdipL8hlZ+kb+81e/BdjVXDK1xlG6L/aB49gQXwiIgjuesrMxOldA47rtC1wGosbkdxGi1NxrDJKWZ8Mgs9ht4cit1FWO1XZqcScJ6cNbP9u5A37ANFyewAeSDBfRyoWhtRId7fO5u8NwNNwbc9647rW9qutRHZlld+HUbI5bdNrv2sQLONtQddNfFZ7H6eSWnkZEbSOAAPiL+66DC7ScwsoaKVxI6RzbMbzdcgO8gVqVM/ecT2knzKt2o2KVz3OcZW9YknrN5m66vyGVn6Rvm1BIfo84lAI5obgTEs0+9uh/DuurimmawbznBrRzJsFr6NhtZw6Rv7zV2ybF69xBdMCWiwsWgjyQZja7tLhMTqOlcXPJIe8Ejc3SLW7bqi4YnSODRxcQPFxtr4q4aXYPO8+kqQzXU7nSe4OF/MKzsnbPaPDd1zGh84FjIW2ue0NJNvPmg8mybJ39GU13gdPMG79uFml2749Y+anKIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIgIiICIiAiIg//9k=",
                            id="unb_logo",
                            style={
                                "height": "60px",
                                "width": "auto",
                                "margin-bottom": "25px",
                            },
                        )
                    ],
                    className="two columns",
                ),
                dhtml.Div(
                    [
                        dhtml.Div(
                            [
                                dhtml.H3(
                                    "Emera and NB Power Research Center for Smart Grid Technologies",
                                    style={
                                        "margin-bottom": "0px",
                                        "text-align": "center"
                                    },
                                ),
                                dhtml.H5(
                                    "EV Research Dashboard", 
                                    style={
                                        "margin-top": "0px",
                                        "text-align": "center"
                                    }
                                ),
                            ]
                        )
                    ],
                    className="eight columns",
                    id="title",
                ),
                dhtml.Div(
                    [
                        dhtml.A(
                            dhtml.Button("Researcher Access", id="researcher-button"),
                            href="http://34.70.91.46:8888",
                        )
                    ],
                    className="two columns",
                    id="button",
                ),
            ],
            id="header",
            className="row flex-display",
            style={"margin-bottom": "25px"},
        ),
        dhtml.Div(
            [
                dhtml.Div(
                    [
                        dhtml.H6(
                            "EV Forecasting",
                            style={
                                "textAlign": "center"
                            }
                        ),
                        dhtml.P(
                            "This dashboard shows a forecast of future impact by an increasing fleet of EVs.",
                            className="description_label"
                        ),
                        dhtml.Div(
                            id="live_clock",
                            children="Date and Time: "
                        ),
                        dcc.Interval(
                            id = "interval-component",
                            interval = 1*1000,
                            n_intervals = 0
                        ),
                        dhtml.Div(
                            id="current_weather"
                        ),
                        dcc.Interval(
                            id = "weather-interval",
                            interval = 60*1000,
                            n_intervals = 0
                        ),
                        dcc.Interval(
                            id = "forecasting-interval",
                            interval = 10*1000,
                            n_intervals = 0
                        ),
                        dhtml.Br(),
                        dhtml.Label(
                            ['Select EV Number to Forecast:'
                            ],
                            style={
                                "textAlign": "center"
                            }
                        ),
                        dcc.Dropdown(
                            id="ev_number_dropdown",
                            options= [{
                                'label': i.title(),
                                'value': i} for i in forecasting_df.columns.values[1:]
                                      
                            ],
                        ),
                    ],
                    className="three columns",
                    id="dashboard-info",
                    style=pretty_container
                ),
                dhtml.Div(
                    [
                        dhtml.Div(
                            [dcc.Graph(id="forecasting_graph")],
                            id="forecastingGraphContainer",
                            style=pretty_container,
                        ),
                    ],
                    id="right-column",
                    className="eight columns",
                    style={
                        "display":"flex",
                        "flex-direction":"column"
                    }
                ),
            ],
            className="row",
            style={
                "display":"flex"
            }
        ),
        dhtml.Div(
            [
                dhtml.Div(
                    [
                        dhtml.H6(
                            "Peak Shaving Algorithm",
                            style={
                                "textAlign":"center"
                            }
                        ),
                        dhtml.P(
                            "Select Optimal Peak-Shaving Load:"
                        ),
                        dcc.Dropdown(
                            id="peak_shaving_dropdown",
                        ),
                        
                    ],
                    className="three columns",
                    id="peak-shaving-info",
                    style=pretty_container,
                ),
                dhtml.Div(
                    [
                        dhtml.Div(
                            [
                                dcc.Graph(
                                    id="peak_shaving_graph"
                                ),
                            ],
                            style=pretty_container
                        )
                    ],
                    className="eight columns",
                    style={
                        "display":"flex",
                        "flex-direction":"column"
                    }
                )
            ],
            className="row"
        )
    ],
)

# Callbacks

In [7]:
# Create callback for current time
@app.callback(Output('live_clock', 'children'),
              [Input("interval-component", "n_intervals")])
def update_time(n):
    return "Date and Time: %s" % (datetime.datetime.now(timezone('Canada/Atlantic')).strftime("%A, %d %B %Y %I:%M:%S %p"))

In [8]:
# Create callback for current temperature and conditions
@app.callback(Output('current_weather', 'children'),
                     [Input("weather-interval", "n_intervals")])
def update_weather(n):
    ec_en = ECData(station_id='NB/s0000250')
    weather = ec_en.conditions
    # current weather and conditions
    global current_temperature, current_conditions
    current_temperature = weather['temperature']['value']
    current_conditions = weather['condition']['value']
    return "Current Weather: %s °C and %s Conditions" % (current_temperature, current_conditions)

In [9]:
# Create callback for forecasting dataframe graph updating with selection
@app.callback(Output('forecasting_graph', 'figure'),
             [Input('ev_number_dropdown', 'value'), Input("forecasting-interval", "n_intervals")])
def update_forecast_df(value, n):
    global forecasting_df
    forecasting_dictionary_object = build_forecasting_df()
    forecasting_df = forecasting_dictionary_object["forecasting_df"]

    forecasting_df.to_dict(orient='records')

    if value == None:
        value = forecasting_df.columns.values[1]
        traces = plotly.graph_objs.Scatter(x = forecasting_df["datetime"],
                                           y = forecasting_df[value])    
    else:
         traces = plotly.graph_objs.Scatter(x = forecasting_df["datetime"],
                                           y = forecasting_df[value])  
    return {'data': [traces], 
            'layout': plotly.graph_objs.Layout(title="EV Forecasting")
           }

# Deploy App

In [ ]:
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
